In [1]:
import numpy as np
import scipy as sp
import scipy.sparse, scipy.io, scipy.optimize
from scipy.special import expit
from tSVD import construct_adjacency_matrix
from lpca import clip_01, lpca_loss_fixed

# Peptides

In [2]:
from torch_geometric.datasets import LRGBDataset

# here we have the name parameter because LRGB has multiple benchmarks.
# for now lets focus on peptides-func
data_peptides = LRGBDataset(name='Peptides-func', root='data', split='train')

/home/tolsi/miniconda3/envs/thesis/lib/python3.10/site-packages/torch_geometric/data/dataset.py:238: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  if osp.exists(f) and torch

In [3]:
np.argmax(np.array(list(map(lambda x: x.x.shape[0], data_peptides))))

np.int64(5238)

In [4]:
adj = sp.sparse.csr_matrix(construct_adjacency_matrix(data_peptides[5238]))

In [5]:
adj.shape

(444, 444)

In [6]:
rank = 128
S = (-1+2*np.random.random(size=rank**2)).reshape((rank, rank))
factor_file_path = 'cora_embed.mat'
n_row, n_col = adj.shape
factors = -1+2*np.random.random(size=(np.sum(adj.shape)*rank)) # initalize uniformly on [-1,+1]

In [7]:
res = scipy.optimize.minimize(lambda x, adj_s, rank: lpca_loss_fixed(x, adj_s, rank, S), x0=factors, 
                              args=(-1 + 2*np.array(adj.todense()), rank), jac=True, method='L-BFGS-B',
                              options={'maxiter':2000}
                             )

1938939.2025545267
1907354.1359793998
1785282.7133254334
1370373.004738891
608589.0665095391
138875.25376257088
85613.42192014385
64196.83643655629
44501.21921260502
23629.618206255032
155223.4096665499
21680.935556351455
470745.45153546514
20350.356791905055
94615.89037719024
19560.15545986606
237104.0478149184
18624.761236145736
76741.58999486562
18373.305103918792
192175.77661641457
17922.367771825542
183669.04038936083
17341.351061838017
148597.2995442837
16691.151615206825
176005.19973503292
16033.381713646595
183761.83269498346
15292.250313747058
130029.17543941467
14369.480885080888
111427.72542794021
13118.19890758055
54554.827440870285
11510.460595207285
22272.85363071424
9372.527146871202
9821.744683182576
7172.688552819959
6255.924871779256
8045.60823542169
5225.122631556696
3986.725575307955
4485.371587878472
3202.4690454645042
4556.464584581024
2584.5278377878185
5285.464759200838
2122.1721980526227
7745.532475429992
1771.435298314026
18831.704387800677
1553.012472016566
3

In [15]:
factors = res.x
U = res.x[:n_row*rank].reshape(n_row, rank)
V = S @ U.T
frob_error_norm = np.linalg.norm(clip_01(U@V) - adj) / sp.sparse.linalg.norm(adj)
print("Frob norm error: ", frob_error_norm)

Frob norm error:  0.0


In [14]:
data = {'U':U, 'V':V}
scipy.io.savemat(factor_file_path, data)

In [39]:
# generate a rank k TSVD factorization of a small sparse matrix adj
def factor_TSVD(adj, k):
    w, v = np.linalg.eigh(np.array(adj.todense()))
    order = np.argsort(np.abs(w))[::-1]
    w = w[order[:k]]
    v = v[:,order[:k]]
    U_tsvd, V_tsvd = v * np.sqrt(np.abs(w))[None,:], (np.sign(w)*np.sqrt(np.abs(w)))[:,None] * v.T
    return U_tsvd, V_tsvd
U_tsvd, V_tsvd = factor_TSVD(adj, rank)

In [40]:
# print the Frobenius errors of the LPCA and TSVD reconstructions, respectively 
print( np.linalg.norm(adj.todense() - expit(U@V)) / sp.sparse.linalg.norm(adj) )
print( np.linalg.norm(adj.todense() - clip_01(U_tsvd@V_tsvd)) / sp.sparse.linalg.norm(adj) )

1.0102797035210955e-08
0.9727326554770129


In [41]:
# print the Frobenius errors of the LPCA and TSVD reconstructions, respectively 
print( np.linalg.norm(adj.todense() - expit(U@V)) / sp.sparse.linalg.norm(adj) )
print( np.linalg.norm(adj.todense() - clip_01(U_tsvd@V_tsvd)) / sp.sparse.linalg.norm(adj) )

1.0102797035210955e-08
0.9727326554770129
